In [5]:
# 1) (one-time) install ollama and helpers
!apt-get update -y
!apt-get install -y pciutils
!curl https://ollama.ai/install.sh | sh
!pip install pyngrok

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Fetched 255 kB in 3s (98.3 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provi

In [6]:
!ollama pull deepseek-coder:6.7b

In [7]:
# start Ollama (serve) in background thread
import os, subprocess, threading, time

def start_ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    os.environ['OLLAMA_KEEP_ALIVE'] = '-1'
    # Start the server (it will run until Colab shuts down)
    subprocess.Popen(["ollama", "serve"], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)

threading.Thread(target=start_ollama, daemon=True).start()
time.sleep(5)
print("Started ollama (give it a minute to load the model on first request).")


Started ollama (give it a minute to load the model on first request).


In [8]:
from pyngrok import ngrok, conf
# set your ngrok authtoken here (replace with your token string)
NGROK_AUTH_TOKEN = "32MHXQiAfZc0zux0IFqtwQbNkSe_58kK19wRiUiNw7keNh9bU"
conf.get_default().auth_token = NGROK_AUTH_TOKEN

# open http tunnel to port 11434 (ollama API)
tunnel = ngrok.connect(11434, "http")
print("Public Ollama URL:", tunnel.public_url)


Public Ollama URL: https://6c4dfa764afd.ngrok-free.app


In [9]:
import requests, json
OLLAMA_URL = "https://834172da0f58.ngrok-free.app"  # replace with your ngrok public url
resp = requests.post(f"{OLLAMA_URL}/api/generate",
                     headers={"Content-Type":"application/json"},
                     json={"model":"codellama:7b","prompt":"Write a short python function that returns Fibonacci numbers","stream": False},
                     timeout=120)
print(resp.status_code)
print(resp.json().get("response","(no response)"))


404
(no response)
